In [19]:
from tqdm.auto import tqdm
from datasets import load_dataset
from transformers import AutoTokenizer, GPT2LMHeadModel

In [21]:
tokenizer = AutoTokenizer.from_pretrained("/home/drdo/Caricatures/models/scan_length_gpt2/checkpoint-58000")

model = GPT2LMHeadModel.from_pretrained("/home/drdo/Caricatures/models/scan_length_gpt2/checkpoint-58000")

model.generation_config.max_length = 256

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [22]:
dataset = load_dataset("scan", "length", trust_remote_code=True)

In [25]:
dataset['test'][2]

{'commands': 'jump around right and jump around left twice',
 'actions': 'I_TURN_RIGHT I_JUMP I_TURN_RIGHT I_JUMP I_TURN_RIGHT I_JUMP I_TURN_RIGHT I_JUMP I_TURN_LEFT I_JUMP I_TURN_LEFT I_JUMP I_TURN_LEFT I_JUMP I_TURN_LEFT I_JUMP I_TURN_LEFT I_JUMP I_TURN_LEFT I_JUMP I_TURN_LEFT I_JUMP I_TURN_LEFT I_JUMP'}

In [26]:
context = "jump around right and jump around left twice"
inputs = tokenizer(context+tokenizer.sep_token, return_tensors="pt")
tokenizer.decode(model.generate(**inputs)[0], skip_special_tokens=False).replace(context+tokenizer.sep_token, '')

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


'I_TURN_RIGHT I_JUMP I_TURN_RIGHT I_JUMP I_TURN_RIGHT I_JUMP I_TURN_RIGHT I_JUMP I_TURN_LEFT I_JUMP I_TURN_LEFT I_JUMP I_TURN_LEFT I_JUMP I_TURN_LEFT I_JUMP I_TURN_LEFT I_JUMP I_TURN_LEFT I_JUMP<|endoftext|>'

In [29]:
test_dataset = dataset['test'] #.select(range(100))

In [30]:
count = 0
model.to("cuda")
bar = tqdm(range(len(test_dataset)))
for example in test_dataset:
    command = example['commands']
    label = example['actions']
    inputs = tokenizer(command+tokenizer.sep_token, return_tensors="pt").to("cuda")
    output = model.generate(**inputs)[0].to("cpu")
    output = tokenizer.decode(output, skip_special_tokens=False).replace(command+tokenizer.sep_token, '')
    output = output.replace(tokenizer.eos_token, '')
    if output == label:
        count += 1
    bar.update(1)

print(count/len(test_dataset))

  0%|          | 0/3920 [00:00<?, ?it/s]

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end gene

0.041581632653061223


In [ ]:
# distilgpt2 : acc = 0.6406025824964132
# distilgpt2 : acc = 0.67 -> beam search (beam size = 3)
# gpt2 : acc = 0.810856049736968, 0.88, 